<a href="https://colab.research.google.com/github/ambresh1/Colab/blob/main/Backtesing_Engine_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Backtesting new Day Sma & 1hr Sma Crossovers

[link text](https://medium.com/@Jachowskii/how-to-build-a-backtesting-engine-in-python-using-pandas-bc8e532a9e95)
**Medium Backtesing Engine using pandas**

Only Change the Entry and Exit Rules and entry & exit Data

In [2]:
import numpy as np
import pandas as pd

In [3]:
infy=pd.read_csv("/content/drive/MyDrive/MyColab/5m data 6months/Infy.csv",index_col=1)
# infy.info()

In [4]:
# Define a Trading Strategy
def SMA(dataset, array, period):
    return array.rolling(period).mean()

In [5]:
sma1d_slow = SMA(infy, infy['Close'], 252) #12*21 of 1hr candels
sma1d_fast = SMA(infy, infy['Close'], 600) #12*50 of 1hr candels
sma_slow = SMA(infy, infy['Close'], 9)
sma_fast = SMA(infy, infy['Close'], 50)

In [6]:
# the entry rules and exit rules of our strategy
def crossover(array1, array2, array3, array4):
  if (array1 > array2).bool and (array3 > array4).bool :
    return array3 > array4
  
def crossunder(array1, array2, array3, array4):
  if (array1 < array2).bool and (array3 < array4).bool :
    return array3 < array4

In [7]:
enter_rules = crossover(sma1d_slow, sma1d_fast, sma_slow, sma_fast)
exit_rules = crossunder(sma1d_slow, sma1d_fast,sma_slow, sma_fast)

In [8]:
check = enter_rules[enter_rules.index == '2021-11-01T09:25:00']
print(check)
check2 = exit_rules[exit_rules.index == '2021-11-01T09:25:00']
print(check2)

Datetime
2021-11-01T09:25:00    False
Name: Close, dtype: bool
Datetime
2021-11-01T09:25:00    False
Name: Close, dtype: bool


In [9]:
# Define a Market Position Function
# turns on if enter_rules is True and exit_rules is False and
# turns off if exit_rules is True.
def marketposition_generator(dataset, enter_rules, exit_rules):

  dataset['enter_rules'] = enter_rules
  dataset['exit_rules'] = exit_rules
  
  status = 0
  mp = []
  for (i, j) in zip(enter_rules, exit_rules):
      if status == 0:
          if i == 1 and j != -1:
              status = 1 
      else:
          if j == -1:
              status = 0
      mp.append(status)
      
  dataset['mp'] = mp
  dataset['mp'] = dataset['mp'].shift(1)
  dataset.iloc[0,2] = 0
  
  return dataset['mp']
# marketposition_generator(infy,enter_rules,exit_rules)

In [10]:
# Define a Backtesting Function
# First of all, we have to define some parameters such as:
# COSTS: fixed costs per trade (i.e. transactions’ fee)
# INSTRUMENT: type of instrument (1 for stocks, 2 for futures, etc.)
# OPERATION_MONEY: initial investment
# DIRECTION: long or short
# ORDER_TYPE: type of order (market, limit, stop, etc.)
# ENTER_LEVEL: entry price

In [11]:
def apply_trading_system(dataset, direction, order_type, enter_level, enter_rules, exit_rules):
    
    dataset['enter_rules'] = enter_rules.apply(lambda x: 1 if x == True else 0)
    dataset['exit_rules'] = exit_rules.apply(lambda x: -1 if x == True else 0)
    dataset['mp'] = marketposition_generator(infy,dataset['enter_rules'], dataset['exit_rules'])
    
    if order_type == "market":
        dataset['entry_price'] = np.where((dataset.mp.shift(1) == 0) & 
                                             (dataset.mp == 1), dataset.Open.shift(1), np.nan)
        if INSTRUMENT == 1:
            dataset['number_of_stocks'] = np.where((dataset.mp.shift(1) == 0) & 
                                                     (dataset.mp == 1), OPERATION_MONEY / dataset.Open, np.nan)
    
    dataset['entry_price'] = dataset['entry_price'].fillna(method='ffill')
    
    if INSTRUMENT == 1:
        dataset['number_of_stocks'] = dataset['number_of_stocks']\
                                        .apply(lambda x: round(x, 0)).fillna(method='ffill')
    
    dataset['events_in'] = np.where((dataset.mp == 1) & (dataset.mp.shift(1) == 0), 'entry', '')
    
    if direction == 'long':
        if INSTRUMENT == 1:
            dataset['open_operations'] = (dataset.Close - dataset.entry_price) * dataset.number_of_stocks
            dataset['open_operations'] = np.where((dataset.mp == 1) & (dataset.mp.shift(-1) == 0),
                                                    (dataset.Open.shift(-1) - dataset.entry_price) * dataset.number_of_stocks - 2 * COSTS, 
                                                     dataset.open_operations)
    else:
        if INSTRUMENT == 1:
            dataset['open_operations'] = (dataset.entry_price - dataset.Close) * dataset.number_of_stocks
            dataset['open_operations'] = np.where((dataset.mp == 1) & (dataset.mp.shift(-1) == 0),
                                                    (dataset.entry_price - dataset.Open.shift(-1)) * dataset.number_of_stocks - 2 * COSTS, 
                                                     dataset.open_operations)
            
    dataset['open_operations'] = np.where(dataset.mp == 1, dataset.open_operations, 0)
    dataset['events_out'] = np.where((dataset.mp == 1) & (dataset.exit_rules == -1), 'exit', '')
    dataset['operations'] = np.where((dataset.exit_rules == -1) & 
                                       (dataset.mp == 1), dataset.open_operations, np.nan)
    dataset['closed_equity'] = dataset.operations.fillna(0).cumsum()
    dataset['open_equity'] = dataset.closed_equity + dataset.open_operations - dataset.operations.fillna(0)
    
    dataset.to_csv('trading_system_export.csv')
    
    return dataset

In [12]:
COSTS = 100
INSTRUMENT = 1 
OPERATION_MONEY = 100000
DIRECTION = "long"
ORDER_TYPE = "market"
ENTER_LEVEL = infy['Open']
trading_system = apply_trading_system(infy, DIRECTION, ORDER_TYPE, ENTER_LEVEL, enter_rules, exit_rules)

In [13]:
net_profit = trading_system['closed_equity'][-1] - OPERATION_MONEY
print(round(net_profit, 2))

-116346.6
